# Imports and Setup

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

# Dataset

In [3]:
# defining constants
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
# using keras to grab a dataset and load to a pandas dataframe
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
# pop species column to use as label
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [7]:
# shape of data
train.shape

(120, 4)

## Input Function

In [8]:
# input function

def input_fn(features, labels, training=True, batch_size=256):
  # convert input to a dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  # shuffle and repeat if in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()
  
  return dataset.batch(batch_size)

## Feature Columns

In [9]:
feature_columns = []
for key in train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key = key))
print(feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Build Model

In [10]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    # Two hidden layers of 30 and 10 nodes each
    hidden_units = [30, 10],
    # The model must choose between 3 classes
    n_classes = 3
)

# Model Training

In [ ]:
classifier.train(
    input_fn = lambda: input_fn(train, train_y, training=True), # We include lambda to avoid creating inner function
    steps=5000)

# Evaluation

In [12]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test, test_y, training=False))

In [17]:
print(f"Test set accuracy: {eval_result['accuracy']}")

Test set accuracy: 0.9333333373069763


# Prediction System

In [18]:
def input_fn(features, batch_size = 256):
  # convert the input to a dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dct in predictions:
  class_id = pred_dct['class_ids'][0]
  probability = pred_dct['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*probability))

Please type numeric values as prompted
SepalLength: 1.3
SepalWidth: 1.4
PetalLength: 3.1
PetalWidth: 3.3
Prediction is "Virginica" (93.1%)
